# Question 1

# Q1.1

lot_type refers to types of lots available at point of data retrieval. Where C refers to car lot, H for heavy vehicles and Y for motorcycles

# 1.2
First, we make an API call for the data from a single specified date & time using the sample code

In [18]:
import requests
import json
import pandas as pd

# Change these values for different dates
year = "2022"
month = "4"
day = "12"
hour = "12"
minute = "30"
second = "0"

site = f'https://api.data.gov.sg/v1/transport/carpark-availability?date_time={year}-{month.zfill(2)}-{day.zfill(2)}T{hour.zfill(2)}%3A{minute.zfill(2)}%3A{second.zfill(2)}'
print(site)
response_API = requests.get(site)
data = response_API.text
data = json.loads(data)
timestamp = data["items"][0]["timestamp"]
print(timestamp)

data = data["items"][0]["carpark_data"]
with open("EE4211data.json", 'w') as fp:
    json.dump(data, fp)
df = pd.read_json("EE4211data.json")
for heading in ("total_lots","lot_type","lots_available"):
    df[heading] = df["carpark_info"].apply(lambda x: x[0][heading])
df = df.drop(["carpark_info"], axis=1)
df

https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T12%3A30%3A00
2022-04-12T12:29:27+08:00


,carpark_number,update_datetime,total_lots,lot_type,lots_available
0,HE12,2022-04-12T12:12:32,105,C,0
1,HLM,2022-04-12T12:12:42,583,C,0
2,RHM,2022-04-12T12:12:32,329,C,106
3,BM29,2022-04-12T12:12:32,97,C,1
4,Q81,2022-04-12T12:12:32,96,C,0
...,...,...,...,...,...
1961,B65M,2022-04-12T12:11:58,679,C,499
1962,B65L,2022-04-12T12:11:58,5,C,0
1963,BE3,2022-04-12T11:30:52,306,C,148
1964,BE3R,2022-04-12T11:30:52,195,C,150


Next, we make a call for the next second.

In [19]:
second = '1'
site = f'https://api.data.gov.sg/v1/transport/carpark-availability?date_time={year}-{month.zfill(2)}-{day.zfill(2)}T{hour.zfill(2)}%3A{minute.zfill(2)}%3A{second.zfill(2)}'
print(site)
response_API = requests.get(site)
data = response_API.text
data = json.loads(data)
timestamp = data["items"][0]["timestamp"]
print(timestamp)

https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T12%3A30%3A01
2022-04-12T12:29:27+08:00


We notice that the updated time has not changed. Therefore, in the following step, we try to figure out the frequency
at which the data values are updated by repeatedly making the call for an interval of 90 seconds. 

In [20]:
def get_timestamp(min_str, sec_str):
    site = f'https://api.data.gov.sg/v1/transport/carpark-availability?date_time={year}-{month.zfill(2)}-{day.zfill(2)}T{hour.zfill(2)}%3A{min_str.zfill(2)}%3A{sec_str.zfill(2)}'
    print(site)
    response_API = requests.get(site)
    data = response_API.text
    data = json.loads(data)
    timestamp = data["items"][0]["timestamp"]
    print(timestamp)
minute = 30
second = 0
for i in range(0,90):
    if second == 59:
        minute += 1
        second = 0
    else:
        second += 1
    min_str = str(minute)
    sec_str = str(second)
    print('min: '+ min_str + ', sec: ' + sec_str)
    get_timestamp(min_str, sec_str)
    

min: 30, sec: 1
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T12%3A30%3A01
2022-04-12T12:29:27+08:00
min: 30, sec: 2
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T12%3A30%3A02
2022-04-12T12:29:27+08:00
min: 30, sec: 3
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T12%3A30%3A03
2022-04-12T12:29:27+08:00
min: 30, sec: 4
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T12%3A30%3A04
2022-04-12T12:29:27+08:00
min: 30, sec: 5
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T12%3A30%3A05
2022-04-12T12:29:27+08:00
min: 30, sec: 6
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T12%3A30%3A06
2022-04-12T12:29:27+08:00
min: 30, sec: 7
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T12%3A30%3A07
2022-04-12T12:29:27+08:00
min: 30, sec: 8
https://api.data.gov.sg/v1/transport/carpark-a

We can see that the updated_time is changed after 1 minute, which means that time updates every 1 minute.


# Q1.3
i) As shown from https://data.gov.sg/dataset/hdb-carpark-information?view_id=398e65ae-e2cb-4312-8651-6e65d6f19ed1&resource_id=139a3035-e624-4f56-b63f-89ae28d4ae4c, there are 2182 carparks in total as stated in the dataset.

ii) This number varies based on the time. for example, at the date and time given in 1.2 at the start, there are 1966 rows which means there are 1966 carparks at that timing. However, given another timing as shown below, at 2022-04-12T15:30, there are 1967 rows now.

In [22]:
year = "2022"
month = "4"
day = "12"
hour = "15"
minute = "30"
second = "0"

site = f'https://api.data.gov.sg/v1/transport/carpark-availability?date_time={year}-{month.zfill(2)}-{day.zfill(2)}T{hour.zfill(2)}%3A{minute.zfill(2)}%3A{second.zfill(2)}'
print(site)
response_API = requests.get(site)
data = response_API.text
data = json.loads(data)
timestamp = data["items"][0]["timestamp"]
print(timestamp)

data = data["items"][0]["carpark_data"]
with open("EE4211data.json", 'w') as fp:
    json.dump(data, fp)
df = pd.read_json("EE4211data.json")
for heading in ("total_lots","lot_type","lots_available"):
    df[heading] = df["carpark_info"].apply(lambda x: x[0][heading])
df = df.drop(["carpark_info"], axis=1)
df

https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T15%3A30%3A00
2022-04-12T15:29:27+08:00


,carpark_number,update_datetime,total_lots,lot_type,lots_available
0,HE12,2022-04-12T15:15:02,105,C,14
1,HLM,2022-04-12T15:15:15,583,C,40
2,RHM,2022-04-12T15:15:02,329,C,202
3,BM29,2022-04-12T15:15:02,97,C,2
4,Q81,2022-04-12T15:15:06,96,C,26
...,...,...,...,...,...
1962,B65L,2022-04-12T15:15:58,5,C,0
1963,JB1,2022-04-12T14:55:17,80,Y,50
1964,BE3,2022-04-12T14:16:56,306,C,159
1965,BE3R,2022-04-12T14:16:56,195,C,145


In [24]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)
display(df)


,carpark_number,update_datetime,total_lots,lot_type,lots_available
0,HE12,2022-04-12T15:15:02,105,C,14
1,HLM,2022-04-12T15:15:15,583,C,40
2,RHM,2022-04-12T15:15:02,329,C,202
3,BM29,2022-04-12T15:15:02,97,C,2
4,Q81,2022-04-12T15:15:06,96,C,26
5,C20,2022-04-12T15:15:16,177,C,0
6,FR3M,2022-04-12T15:15:21,228,C,134
7,C32,2022-04-12T15:14:58,289,C,174
8,C6,2022-04-12T15:15:20,332,C,155
9,TG2,2022-04-12T15:15:00,273,C,128


iii) Using the date and timing in 1.3ii), we observed that there are some carpark with malfunctioning sensors. For example, for carpark RCB from column index 40, the data has not been updated since 2017-3-31T12:00:21, this suggests that the carpark sensor was malfunctioned from that time onwards.